# Credit Approval ML - Exploração e Modelagem

Notebook para exploração de dados, treinamento e avaliação do modelo de predição de aprovação de crédito.

## 1. Importações e Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Configurar visualizações
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Importar modelo local
from src.models.credit_model import CreditApprovalModel
from src.utils.logger import setup_logging, get_logger

setup_logging()
logger = get_logger(__name__)

## 2. Gerar Dados Sintéticos

In [ ]:
# Setar seed para reprodutibilidade
np.random.seed(42)

# Gerar dados sintéticos
n_samples = 1500

X = pd.DataFrame({
    'age': np.random.randint(18, 75, n_samples),
    'income': np.random.randint(20000, 200000, n_samples),
    'credit_score': np.random.randint(300, 850, n_samples),
    'loan_amount': np.random.randint(5000, 100000, n_samples),
    'employment_years': np.random.randint(0, 50, n_samples),
    'existing_debts': np.random.randint(0, 50000, n_samples),
})

# Criar target com regra realista
y = (
    (X['credit_score'] > 600)
    & (X['income'] > X['loan_amount'] * 0.2)
    & (X['age'] > 21)
).astype(int)

print(f"Dataset shape: {X.shape}")
print(f"Distribuição do target:")
print(y.value_counts())
print(f"\nPercentual de aprovados: {(y == 1).sum() / len(y) * 100:.2f}%")

## 3. Exploração de Dados (EDA)

In [ ]:
# Estatísticas básicas
X.describe()

In [ ]:
# Correlação entre features
correlation_matrix = X.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Matriz de Correlação das Features')
plt.tight_layout()
plt.show()

In [ ]:
# Distribuição das features principais
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
features = ['age', 'income', 'credit_score', 'loan_amount', 'employment_years', 'existing_debts']

for idx, feature in enumerate(features):
    ax = axes[idx // 3, idx % 3]
    ax.hist(X[feature], bins=30, edgecolor='black', alpha=0.7)
    ax.set_title(f'Distribuição: {feature.capitalize()}')
    ax.set_xlabel(feature)
    ax.set_ylabel('Frequência')

plt.tight_layout()
plt.show()

In [ ]:
# Médias por classe (aprovado vs reprovado)
comparison_df = X.copy()
comparison_df['approved'] = y

print("Médias por Status:")
print(comparison_df.groupby('approved').mean().round(2))

## 4. Splitting e Treinamento

In [ ]:
# Dividir dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Treino: {X_train.shape}")
print(f"Teste: {X_test.shape}")

In [ ]:
# Treinar modelo usando classe local
model = CreditApprovalModel()
metrics = model.train(X_train, y_train)

print(f"\nMétricas de Treinamento:")
for key, value in metrics.items():
    print(f"  {key}: {value}")

## 5. Avaliação no Conjunto de Teste

In [ ]:
# Predições
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Métricas
test_accuracy = (y_pred == y_test.values).mean()
auc_score = roc_auc_score(y_test, y_proba)

print(f"Acurácia no Teste: {test_accuracy:.4f}")
print(f"AUC-ROC: {auc_score:.4f}")
print(f"\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['Reprovado', 'Aprovado']))

In [ ]:
# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Reprovado', 'Aprovado'],
            yticklabels=['Reprovado', 'Aprovado'])
plt.title('Matriz de Confusão')
plt.ylabel('Real')
plt.xlabel('Predito')
plt.tight_layout()
plt.show()

In [ ]:
# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_proba)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc_score:.4f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Feature Importance

In [ ]:
# Extrair importância das features
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature', palette='viridis')
plt.title('Importância das Features')
plt.xlabel('Importância')
plt.tight_layout()
plt.show()

print(feature_importance)

## 7. Salvar Modelo para Produção

In [ ]:
from pathlib import Path

# Salvar modelo
model_dir = Path('models_trained')
model_dir.mkdir(exist_ok=True)

model.save(
    str(model_dir / 'credit_model.pkl'),
    str(model_dir / 'scaler.pkl')
)

print(f"✓ Modelo salvo em {model_dir}")

## 8. Teste com Exemplos

In [ ]:
# Teste com alguns exemplos
test_cases = pd.DataFrame([
    {
        'age': 35,
        'income': 50000,
        'credit_score': 750,
        'loan_amount': 20000,
        'employment_years': 8,
        'existing_debts': 5000,
        'description': 'Candidato com bom perfil'
    },
    {
        'age': 25,
        'income': 25000,
        'credit_score': 550,
        'loan_amount': 30000,
        'employment_years': 1,
        'existing_debts': 15000,
        'description': 'Candidato com risco alto'
    },
    {
        'age': 45,
        'income': 100000,
        'credit_score': 820,
        'loan_amount': 50000,
        'employment_years': 20,
        'existing_debts': 0,
        'description': 'Candidato premium'
    }
])

for _, case in test_cases.iterrows():
    desc = case.pop('description')
    X_case = pd.DataFrame([case])
    
    pred = model.predict(X_case)[0]
    proba = model.predict_proba(X_case)[0][1]
    
    status = '✓ APROVADO' if pred else '✗ REPROVADO'
    print(f"{desc}:")
    print(f"  {status}")
    print(f"  Probabilidade: {proba:.2%}")
    print()

---
**Próximos passos:**
1. Executar `make train-model` para persistir o modelo
2. Executar `make run` para iniciar a API
3. Testar endpoints em http://localhost:8000/docs